![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Disponibilización de modelos

En este notebook aprenderá a guardar un modelo y a disponibilizarlo como una API con la librería Flask. Una API (interfaz de programación de aplicaciones) es un conjunto de definiciones y protocolos que permiten que servicios, en este caso modelos, retornen resultados y respuestas sin necesidad de saber cómo están implementados.

## Instrucciones Generales:

Este notebook esta compuesto por dos secciones. En la primera secciónn, usted beberá entrenar y guardar (exportar) un modelo de random forest para predecir si una URL es phishing (fraudulenta) o no. En la segunda parte, usará el modelo entrenado y lo disponibilizara usando la libreria *Flask*. En el siguente paper puede conocer más detalles de la base de datos que usaremos y del problema: *A. Correa Bahnsen, E. C. Bohorquez, S. Villegas, J. Vargas, and F. A. Gonzalez, “Classifying phishing urls using recurrent neural networks,” in Electronic Crime Research (eCrime), 2017 APWG Symposium on. IEEE, 2017, pp. 1–8*. https://albahnsen.files.wordpress.com/2018/05/classifying-phishing-urls-using-recurrent-neural-networks_cameraready.pdf
  
Para realizar la actividad, solo siga las indicaciones asociadas a cada celda del notebook. 


## Importar base de datos y librerías

In [2]:
#!pip install -r requirements.txt

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Importación librerías
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import joblib

import os
os.chdir('..')

In [ ]:
# Carga de datos de archivos .csv
data = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/phishing.csv')
data.head()

,url,phishing
0,http://www.subalipack.com/contact/images/sampl...,1
1,http://fasc.maximecapellot-gypsyjazz-ensemble....,1
2,http://theotheragency.com/confirmer/confirmer-...,1
3,http://aaalandscaping.com/components/com_smart...,1
4,http://paypal.com.confirm-key-21107316126168.s...,1


In [7]:

data.tail()

,url,phishing
39995,http://www.diaperswappers.com/forum/member.php...,0
39996,http://posting.bohemian.com/northbay/Tools/Ema...,0
39997,http://www.tripadvisor.jp/Hotel_Review-g303832...,0
39998,http://www.baylor.edu/content/services/downloa...,0
39999,http://www.phinfever.com/forums/viewtopic.php?...,0


## Codificar variables categóricas
Relizar preprocesamiento de texto (URLs) para crear variables predictoras:

In [8]:
# Creación de columnas binarias que indican si la URL contiene la palabra clave (keywords)
keywords = ['https', 'login', '.php', '.html', '@', 'sign']
for keyword in keywords:
    data['keyword_' + keyword] = data.url.str.contains(keyword).astype(int)

# Definición de la variable largo de la URL
data['lenght'] = data.url.str.len() - 2

# Definición de la variable largo del dominio de la URL
domain = data.url.str.split('/', expand=True).iloc[:, 2]
data['lenght_domain'] = domain.str.len()

# Definición de la variable binaria que indica si es IP
data['isIP'] = (domain.str.replace('.', '') * 1).str.isnumeric().astype(int)

# Definicón de la variable cuenta de 'com' en la URL
data['count_com'] = data.url.str.count('com')

data.head()

,url,phishing,keyword_https,keyword_login,keyword_.php,keyword_.html,keyword_@,keyword_sign,lenght,lenght_domain,isIP,count_com
0,http://www.subalipack.com/contact/images/sampl...,1,0,0,0,0,0,0,47,18,0,1
1,http://fasc.maximecapellot-gypsyjazz-ensemble....,1,0,0,0,0,0,0,73,41,0,0
2,http://theotheragency.com/confirmer/confirmer-...,1,0,0,0,0,0,0,92,18,0,1
3,http://aaalandscaping.com/components/com_smart...,1,0,0,0,0,0,0,172,18,0,3
4,http://paypal.com.confirm-key-21107316126168.s...,1,0,0,0,0,0,0,90,50,0,1


In [9]:
# Separación de variables predictoras (X) y variable de interes (y)
X = data.drop(['url', 'phishing'], axis=1)
y = data.phishing

## Entrenar y guardar el modelo

In [10]:
# Definición de modelo de clasificación Random Forest
clf = RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=3)
cross_val_score(clf, X, y, cv=10)

array([0.76125, 0.758  , 0.75025, 0.76025, 0.765  , 0.7675 , 0.754  ,
       0.75475, 0.752  , 0.756  ])

In [11]:
# Entrenamiento del modelo de clasificación Random Forest
clf.fit(X, y)

RandomForestClassifier(max_depth=3, n_jobs=-1)

In [20]:
import os
print(os.getcwd())

c:\Users\madelgado\Documents\GitHub


In [23]:
# Exportar modelo a archivo binario .pkl
joblib.dump(clf, 'model_deployment/phishing_clf.pkl', compress=3)

['model_deployment/phishing_clf.pkl']

In [26]:
#creo este archivo para que me deje ejecutar
open("model_deployment/__init__.py", 'a').close()


In [36]:
# model_deployment/m09_model_deployment.py

import joblib
import pandas as pd
import re

# Cargar el modelo previamente entrenado
clf = joblib.load('model_deployment/phishing_clf.pkl')

# Lista de keywords usadas como features
keywords = ['https', 'login', '.php', '.html', '@', 'sign']

# Función que transforma una URL en el mismo conjunto de features que usaste para entrenar
def extract_features(url):
    domain = url.split('/')[2] if len(url.split('/')) > 2 else ''

    features = {}

    # Variables binarias de keywords
    for keyword in keywords:
        features['keyword_' + keyword] = int(keyword in url)

    # Largo de la URL (menos 2 como en tu entrenamiento)
    features['lenght'] = len(url) - 2

    # Largo del dominio
    features['lenght_domain'] = len(domain)

    # Variable binaria si es IP (todo el dominio son dígitos después de eliminar puntos)
    features['isIP'] = int(re.sub(r'\.', '', domain).isdigit())

    # Cuenta de ocurrencias de "com"
    features['count_com'] = url.count('com')

    return features

# Función principal de predicción
def predict_proba(url):
    features = extract_features(url)
    df = pd.DataFrame([features])
    return clf.predict_proba(df)


ValueError: node array from the pickle has an incompatible dtype:
- expected: {'names': ['left_child', 'right_child', 'feature', 'threshold', 'impurity', 'n_node_samples', 'weighted_n_node_samples', 'missing_go_to_left'], 'formats': ['<i8', '<i8', '<i8', '<f8', '<f8', '<i8', '<f8', 'u1'], 'offsets': [0, 8, 16, 24, 32, 40, 48, 56], 'itemsize': 64}
- got     : [('left_child', '<i8'), ('right_child', '<i8'), ('feature', '<i8'), ('threshold', '<f8'), ('impurity', '<f8'), ('n_node_samples', '<i8'), ('weighted_n_node_samples', '<f8')]

In [34]:
import os

# Cambiar el directorio de trabajo
os.chdir(r'C:\Users\madelgado\Documents\GitHub\MIAD_ML_NLP_2025')

# Verificar que estás en el directorio correcto
print("Directorio actual:", os.getcwd())


Directorio actual: C:\Users\madelgado\Documents\GitHub\MIAD_ML_NLP_2025


In [35]:
# Importar modelo y predicción
from model_deployment.phishing_clf import predict_proba

# Predicción de probabilidad de que un link sea phishing
predict_proba('http://www.vipturismolondres.com/com.br/?atendimento=Cliente&/LgSgkszm64/B8aNzHa8Aj.php')

ModuleNotFoundError: No module named 'model_deployment.phishing_clf'

## Disponibilizar modelo con Flask

Para esta sección del notebook instale las siguientes librerías *!pip install flask* y *!pip install flask_restplus*.

In [28]:
# Importación librerías
from flask import Flask
from flask_restx import Api, Resource, fields

In [29]:
app = Flask(__name__)

# Definición API Flask
api = Api(
    app, 
    version='1.0', 
    title='Phishing Prediction API',
    description='Phishing Prediction API')

ns = api.namespace('predict', 
     description='Phishing Classifier')

# Definición argumentos o parámetros de la API
parser = ns.parser()
parser.add_argument(
    'URL', 
    type=str, 
    required=True, 
    help='URL to be analyzed', 
    location='args')

resource_fields = api.model('Resource', {
    'result': fields.String,
})

In [30]:
# Definición de la clase para disponibilización
@ns.route('/')
class PhishingApi(Resource):

    @ns.doc(parser=parser)
    @ns.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()
        
        return {
         "result": predict_proba(args['URL'])
        }, 200

In [31]:
# Ejecución de la aplicación que disponibiliza el modelo de manera local en el puerto 5000
app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.9:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Apr/2025 11:53:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 11:53:24] "GET /swaggerui/swagger-ui.css HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 11:53:24] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 11:53:24] "GET /swaggerui/droid-sans.css HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 11:53:24] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 11:53:24] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 11:53:24] "GET /swaggerui/favicon-32x32.png HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 11:55:14] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 11:55:14] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 11:55:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 11:55:19] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
12

El modelo debe haber quedado disponibilizado en el puerto 5000. Para predecir la probabilidad de que una URL sea fraudulenta (phishing) copie en la barra de busqueda de su navegador la siguiente dirección (http://localhost:5000/predict/?URL=) y agregregue al final de esta la URL que desee precir. Por ejemplo, al copiar la URL http://localhost:5000/predict/?URL=http://consultoriojuridico.co/pp/www.paypal.com/, la API retornará la probabilidad de que la URL http://consultoriojuridico.co/pp/www.paypal.com/ sea phishing.